In [9]:
import os
import pandas as pd
import spacy

In [10]:
filename = "synthetic_ner_health_data.csv"
filepath = os.path.abspath(filename)

df = pd.read_csv(filepath)

In [11]:
nlp = spacy.load("en_ner_bc5cdr_md")
print(nlp.get_pipe("ner").labels)

('CHEMICAL', 'DISEASE')


In [12]:
nlp = spacy.load("en_ner_bc5cdr_md")
# nlp.pipe_names

# ent = doc.ents[0]

# print(dir(ent))

def annotating(text):
    doc = nlp(text)

    annotation_dict = {}

    symptons = ["chest pain", "cough", "dizziness", "fatigue", "fever", "headache", "joint pain", "nausea", "shortness of breath", "sore throat"]


    for ent in doc.ents:
        if ent.text.lower() in symptons and ent.label_ == "DISEASE":
            annotation_dict[ent.text.lower()] = {"start":ent.start_char, "end":ent.end_char, "label":"SYMPTON"}
        else:
            annotation_dict[ent.text.lower()] = {"start":ent.start_char, "end":ent.end_char, "label":ent.label_}

    return annotation_dict


df["annotations"] = df["text"].apply(annotating)

In [13]:
nlp = spacy.blank("en")
def tagging(row):
    
    doc = nlp(row["text"].lower())

    tokens = [token.text for token in doc]

    ner_tags = []

    for token in doc:
        tag="O"

        for value in row["annotations"].values():
            char_start = value["start"]
            char_end = value["end"]
            label = value["label"][:4]

            if token.idx == char_start:
                tag = f"B-{label}"
            elif char_start<token.idx<char_end:
                tag =f"I-{label}"

        ner_tags.append(tag)

    return tokens,ner_tags

df[["tokens","ner_tags"]] = df[["text","annotations"]].apply(tagging, axis=1,result_type='expand')

In [14]:
tags_dict = {"O":0, "B-CHEM":1, "I-CHEM":2, "B-DISE":3, "I-DISE":4, "B-SYMP":5, "I-SYMP":6}

def labels(ner_tags):
    labels_list = []
    for ner_tag in ner_tags:
        labels_list.append(tags_dict[ner_tag])
    return labels_list

df["labels"] = df["ner_tags"].apply(labels)

In [15]:
filename = "TokenCLS_Dataset_with_ner_tags.csv"
filepath = os.path.abspath(filename)
df[["tokens","ner_tags","labels"]].to_csv(filepath, index=False)

In [16]:
filename = "TokenCLS_Dataset.csv"
filepath = os.path.abspath(filename)
df[["tokens","labels"]].to_csv(filepath, index=False)